In [1]:
"""
Feature: urine_culture_performed
Flag if urine culture was performed from microbiologyevents
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load microbiology events
microbiology_events = pd.read_csv(
    os.path.join(hosp_path, "microbiologyevents.csv"),
    usecols=["subject_id", "hadm_id", "spec_type_desc", "test_name"]
)

# Filter urine culture
urine_culture_df = microbiology_events[
    (microbiology_events["spec_type_desc"].str.upper() == "URINE") &
    (microbiology_events["test_name"].str.upper() == "URINE CULTURE")
].copy()

# Admission-level flag
urine_culture_flag = (
    urine_culture_df[["subject_id", "hadm_id"]]
    .drop_duplicates()
    .assign(urine_culture_performed=True)
)

# Merge
df = df.merge(urine_culture_flag, on=["subject_id", "hadm_id"], how="left")

# Convert to boolean
df["urine_culture_performed"] = df["urine_culture_performed"].notna().astype(bool)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nFeature 'urine_culture_performed' added.")
print(f"Value counts:\n{df['urine_culture_performed'].value_counts()}")
print(f"Total rows: {len(df)}")
print(f"True count: {df['urine_culture_performed'].sum()}")
print(f"False count: {(~df['urine_culture_performed']).sum()}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")




Feature 'urine_culture_performed' added.
Value counts:
urine_culture_performed
False    104975
True      53045
Name: count, dtype: int64
Total rows: 158020
True count: 53045
False count: 104975
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 87)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "urine_culture_performed"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)